In [1]:
import math, os, sys
import json
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.offline as py
from pathlib import Path
from datetime import datetime
from ipywidgets import interactive, HBox, VBox
from ipywidgets import widgets
#py.init_notebook_mode()

sys.path.append('/home/eddyod/programming/activebrainatlas')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "activebrainatlas.settings")
import django
django.setup()

In [2]:
from neuroglancer.models import UrlModel, LayerData, CenterOfMass, Structure
from brain.models import Animal

In [ ]:
def find_values(id, json_repr):
    results = []
    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results

In [12]:
urlModel = UrlModel.objects.get(pk=200)
df = urlModel.points
df.query('Layer in ("premotor", "PM nucleus")').head(200)

,Layer,Description,X,Y,Section
0,premotor,,46968,18713,244
0,PM nucleus,,40798,18919,314
1,PM nucleus,,40277,18955,314
2,PM nucleus,,40047,19538,314
3,PM nucleus,,41099,20501,314
...,...,...,...,...,...
194,PM nucleus,,45963,18365,185
195,PM nucleus,,46456,18634,185
196,PM nucleus,,45967,18978,185
197,PM nucleus,,46476,18254,185


In [34]:
img_width = 65000
img_height = 36000

id = 200
urlModel = UrlModel.objects.get(pk=id)
df = urlModel.points
df = df.query('Layer in ("premotor", "PM nucleus")')
df.reset_index(inplace=True)
df['ID'] = df.index
cols = ['ID', 'Layer','X','Y', 'Section']
section = df['Section'].min()
initial_df = df.loc[df['Section'] == section]
section = str(section).zfill(3)
source = f"https://activebrainatlas.ucsd.edu/data/DK52/www/{section}.png"
fig = go.FigureWidget([go.Scatter(y = initial_df['Y'], x = initial_df['X'], mode = 'markers')])
fig.add_layout_image(
        dict(
        x=0,
        sizex=img_width,
        y=0,
        sizey=img_height,
        xref="x",
        yref="y",
        yanchor="top",
        opacity=0.5,
        layer="below",
        name="showme",
        source=source,
        sizing="contain")
)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_xaxes(range=[0, img_width])
fig.update_yaxes(range=[0, img_height],scaleanchor="x")
fig.update_layout(template="plotly_white")
fig['layout']['yaxis']['autorange'] = "reversed"

scatter = fig.data[0]

scatter.marker.opacity = 1
scatter.marker.size = 6

section_selector = widgets.Dropdown(
    options=list(df['Section'].sort_values().unique()),
    value=df['Section'].iloc[0],
    description='Section:',
)

def response(change):
    filter_list = [s for s in df['Section'] == section_selector.value]
    tmp_df = df[filter_list]
        
    x = tmp_df['X']
    y = tmp_df['Y']
    with fig.batch_update():
        fig.layout.xaxis.title = "X"
        fig.layout.yaxis.title = "Y"
        section = str(section_selector.value).zfill(3)
        fig.layout.images[0].source=f"https://activebrainatlas.ucsd.edu/data/DK52/www/{section_selector.value}.png"
        scatter.x = x
        scatter.y = y
    with t.batch_update():
        t.layout.title = f"{len(tmp_df)} points"
        t.data[0].cells.values = [tmp_df[col] for col in tablecols]

section_selector.observe(response, names="value")

tablecols = ['Layer', 'X', 'Y', 'Section']
t = go.FigureWidget([go.Table(
    header=dict(values=tablecols,
                fill = dict(color='#C2D4FF'),
                align = ['left'] * 5),
    cells=dict(values=[df[col] for col in tablecols],
               fill = dict(color='#F5F8FF'),
               align = ['left'] * 5))])

def selection_fn(trace,points,selector):
    t.layout.title = f"You have selected {len(points.point_inds)} points"
    t.data[0].cells.values = [df.loc[points.point_inds][col] for col in tablecols]

scatter.on_selection(selection_fn)

figure_container = widgets.VBox([fig, t])
widgets.VBox([section_selector, figure_container])

In [ ]:
#df2 = pd.read_csv('https://raw.githubusercontent.com/jonmmease/plotly_ipywidget_notebooks/master/notebooks/data/cars/cars.csv')
#df2.select_dtypes('int64').columns
df['Section'].sort_values().unique()

In [ ]:
df.head()